MMDetection3D is an open source object detection toolbox based on PyTorch, towards the next-generation platform for general 3D detection. It is a part of the OpenMMLab project. @misc{mmdet3d2020,
    title={{MMDetection3D: OpenMMLab} next-generation platform for general {3D} object detection},
    author={MMDetection3D Contributors},
    howpublished = {\url{https://github.com/open-mmlab/mmdetection3d}},
    year={2020}
}

In [ ]:
# Redução e limpeza da nuvem utilizando a biblioteca open3d.
import open3d as o3d

point_cloud = o3d.io.read_point_cloud("./quadrante-quatro.ply") # caminho para o arquivo de entrada da nuvem.
pcd_filtered = point_cloud.voxel_down_sample(voxel_size=0.005) # voxel_size - variável de controle.
pcd_filter, ind = pcd_filtered.remove_statistical_outlier(nb_neighbors=20, std_ratio=1) #nb_neighbors e std_ratio - variáveis de controle.

o3d.io.write_point_cloud("./output.ply", pcd_filter) # caminho para o arquivo de saída da nuvem reduzida.

In [ ]:
# Visualização da Nuvem utilizando a biblioteca "Trimesh".
import trimesh

cloud = trimesh.load("./quadrante-quatro.ply") # caminho para o arquivo a ser visualizado.
pcd = trimesh.PointCloud(cloud.vertices, cloud.colors) # inclui os vértices e cores a nuvem a ser visualizada.
scene = trimesh.Scene([pcd]) # cria uma cena contendo a nuvem carregada.
scene.show() # comando para visualização da nuvem.

In [1]:
# Preparação da nuvem para inferência.
import numpy as np
import pandas as pd
from plyfile import PlyData

# Método de apoio para inspeção da nuvem.
def inspect_ply(input_path):
    plydata = PlyData.read(input_path)
    data = plydata.elements[0].data
    property_names = data[0].dtype.names
    print(f"Number of points: {len(data)}")
    print(f"Properties per point: {property_names}")

# inspect_ply('./input.ply') # caminho do arquivo da nuvem a ser inspecionada.

# Converte a nuvem de entrada com extensão ".ply" para ".bin" (extensão necessária para realização da inferência).
# O método abaixo foi adaptado a partir das implementações da biblioteca MMDetection3D para converter corretamente 
# nuvens com qualquer quantidade de pontos.
def convert_ply(input_path, output_path):
    plydata = PlyData.read(input_path)  # leitura do arquivo
    data = plydata.elements[0].data  # obtenção dos dados
    data_pd = pd.DataFrame(data)  # converte para DataFrame.
    
    # Especifica as propriedades a serem utilizadas.
    properties_to_use = ['x', 'y', 'z', 'nx', 'ny', 'nz']
    num_points = data_pd.shape[0]
    num_properties = len(properties_to_use)
    
    data_np = np.zeros((num_points, num_properties), dtype=np.float64)  # inicializa array para armazenar dados.
    
    for i, name in enumerate(properties_to_use):  # ler dados por propriedade.
        data_np[:, i] = data_pd[name]
    
    # salva o array em um arquivo binário.
    data_np.astype(np.float32).tofile(output_path)

convert_ply('./quadrante-quatro.ply', './output.bin') # caminho da nuvem de entrada / caminho para nuvem de saída.

### Executando inferência para Detecção com modelo pré-treinado.
```python ./mmdetection3d/demo/pcd_demo.py ./test.bin ./checkpoints/votenet_8xb16_sunrgbd-3d.py ./checkpoints/votenet_16x8_sunrgbd-3d-10class_20210820_162823-bf11f014.pth --pred-score-thr 0.99```

- ```./mmdetection3d/demo/pcd_demo.py``` - caminho para o algoritmo de demonstração.
- ```./test.bin``` - caminho para nuvem a ser inferida.
- ```./checkpoints/votenet_8xb16_sunrgbd-3d.py``` - caminho para modelo de inferencia.
- ```./checkpoints/votenet_16x8_sunrgbd-3d-10class_20210820_162823-bf11f014.pth``` - caminho para checkpoint.
- ```--pred-score-thr 0.99``` - define score mínimo para inferência.

In [ ]:
import json
import numpy as np
import open3d as o3d

# Carregar nuvem de pontos a partir dos dados salvos
point_cloud = o3d.io.read_point_cloud("quadrante-quatro.ply")

# Carregar dados da predição da bounding box a partir do arquivo JSON
with open("./outputs/preds/output.json", "r") as f:
    data = json.load(f)

# Extrair todas as bounding boxes
bboxes = data["bboxes_3d"]
total_bboxes = len(bboxes)

# Converter a nuvem de pontos para numpy arrays
points = np.asarray(point_cloud.points)
colors = np.asarray(point_cloud.colors)

# Função para verificar se pontos estão dentro da bounding box orientada
def are_points_inside_bbox(points, bbox):
    center = np.array(bbox[0:3])
    dim = np.array(bbox[3:6])
    yaw = np.zeros(3)
    yaw[2] = bbox[6]

    # Calcular matriz de rotação a partir da orientação da bounding box
    rot_mat = o3d.geometry.get_rotation_matrix_from_xyz(yaw)
    center[2] += dim[2] / 2

    # Transformar os pontos para o sistema de coordenadas da bounding box alinhada
    points_local = np.dot(points - center, rot_mat.T)
    return np.all(np.abs(points_local) <= dim / 2, axis=1)

# Inicializar máscara para manter todos os pontos inicialmente
mask = np.ones(len(points), dtype=bool)

# Iterar sobre todas as bounding boxes
for idx, bbox in enumerate(bboxes):
    print(f"Processando bounding box {idx + 1} de {total_bboxes}")
    mask &= ~are_points_inside_bbox(points, bbox)

# Aplicar a máscara para filtrar os pontos e suas cores correspondentes
filtered_points = points[mask]
filtered_colors = colors[mask]

# Criar uma nova nuvem de pontos com os pontos filtrados e suas cores correspondentes
new_point_cloud = o3d.geometry.PointCloud()
new_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)
new_point_cloud.colors = o3d.utility.Vector3dVector(filtered_colors)

# Salvar a nova nuvem de pontos em um único arquivo
o3d.io.write_point_cloud("result.ply", new_point_cloud)
print("Salvo: result.ply")
print(f"Total de bounding boxes processadas: {total_bboxes}")